In [162]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [163]:
# Load CSV file
csv_path = "/content/gdrive/My Drive/labx7.csv"
image_folder_path = "/content/gdrive/My Drive/dataset final"

df = pd.read_csv(csv_path)

# Define target variable
target = df['Text Font Style']

# Resize and convert images to numpy arrays
image_data = []
for image_name in df['Image Name']:
    image_path = f"{image_folder_path}/{image_name}"
    image = load_img(image_path, target_size=(224, 224))
    image_array = img_to_array(image)
    image_data.append(image_array)

image_data = np.array(image_data)

In [164]:
# Map each image with the target variable
# Drop Image Name column
df = df.drop(columns=['Image Name'])

# Label encode target variable
encoder = LabelEncoder()
target_encoded = encoder.fit_transform(target)
print("Encoding mapping:")
for idx, label in enumerate(encoder.classes_):
    print(f"{label} -> {idx}")

Encoding mapping:
Excellent -> 0
Good -> 1
None -> 2
Poor -> 3


In [165]:

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(image_data, target_encoded, test_size=0.2, random_state=42)


In [170]:
# Define hyperparameters
batch_size = 32
epochs = 5
learning_rate = 0.001

# Define CNN model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(4, activation='softmax')
])

In [171]:

# Compile the model
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [172]:
# Train the model
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)


Epoch 1/5
26/26 [==============================] - 3s 70ms/step - loss: 124.8441 - accuracy: 0.6445 - val_loss: 0.9761 - val_accuracy: 0.8551
Epoch 2/5
26/26 [==============================] - 2s 61ms/step - loss: 0.9726 - accuracy: 0.8464 - val_loss: 0.9178 - val_accuracy: 0.8599
Epoch 3/5
26/26 [==============================] - 2s 61ms/step - loss: 0.8763 - accuracy: 0.8670 - val_loss: 0.8926 - val_accuracy: 0.8551
Epoch 4/5
26/26 [==============================] - 1s 57ms/step - loss: 0.7206 - accuracy: 0.8863 - val_loss: 0.7798 - val_accuracy: 0.8599
Epoch 5/5
26/26 [==============================] - 1s 55ms/step - loss: 0.6265 - accuracy: 0.8815 - val_loss: 0.7868 - val_accuracy: 0.8744


In [173]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_acc)


9/9 [==============================] - 0s 20ms/step - loss: 0.8601 - accuracy: 0.8224
Test Accuracy: 0.8223938345909119


In [174]:

# Save the model
model.save("/content/gdrive/My Drive/Text Font Style(82).h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [128]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model
from google.colab import drive

drive.mount('/content/gdrive')

# Load the trained model
model_path = "/content/gdrive/My Drive/Overlay Color Scheme(71).h5"
model = load_model(model_path)

# Define function to predict intensity for an image
def predict_intensity(image_path):
    # Load and preprocess the image
    image = load_img(image_path, target_size=(224, 224))
    image_array = img_to_array(image)
    image_array = np.expand_dims(image_array, axis=0)
    # Make prediction using the model
    prediction = model.predict(image_array)
    predicted_intensity = np.argmax(prediction)
    return predicted_intensity

# Example usage
image_path = "/content/gdrive/My Drive/dataset final/Utility Apps (63).png"
predicted_intensity = predict_intensity(image_path)
print("Predicted Intensity:", predicted_intensity)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
1/1 [==============================] - 0s 82ms/step
Predicted Intensity: 1


In [ ]:
Utility Apps (9).png
bad (17).png

In [4]:
model.save("/content/gdrive/My Drive/cnn_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [40]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Load the saved model
model_path = "/content/gdrive/My Drive/overlay_intensity1(71).h5"
model = load_model(model_path)

# Define function to predict intensity for a given image
def predict_intensity(image_path):
    # Load and preprocess the image
    image = load_img(image_path, target_size=(224, 224))
    image_array = img_to_array(image)
    image_array = np.expand_dims(image_array, axis=0)

    # Make prediction
    prediction = model.predict(image_array)

    # Decode prediction
    intensity_classes = ['Low', 'Moderate', 'High', 'None']
    predicted_class = np.argmax(prediction)
    intensity = intensity_classes[predicted_class]

    return intensity

# Path to the image for prediction
image_path = "/content/gdrive/My Drive/dataset final/bad (200).png"

# Predict intensity for the image
predicted_intensity = predict_intensity(image_path)
print("Predicted Overlay Intensity:", predicted_intensity)


1/1 [==============================] - 0s 82ms/step
Predicted Overlay Intensity: Low


In [7]:
encoder = LabelEncoder()
target_encoded = encoder.fit_transform(target)
print(target_encoded)

[0 0 0 ... 1 1 2]
